In [1]:
from typing import List, Tuple
import snoop

In [2]:
board_old = [
    [0, 1, 0],
    [0, 0, 2],
    [0, 0, 0]]
board_new = [
    [0, 0, 0],
    [0, 0, 0],
    [1, 0, 0]]
# 0 = empty, 1 = X, 2 = O

In [3]:
def get_latest_move(prev_state: List[List[int]], current_state: List[List[int]])-> Tuple[int, int]:
    for i in range(0, len(prev_state)):
        for j in range(0, len(current_state[0])):
            if prev_state[i][j] != current_state[i][j]:
                return (i,j)

In [4]:
assert get_latest_move(board_old, board_new) == (0,1)

In [89]:
def adjacency_check(i, j, matrix, num_neighbors=2):
    min_i = 0 
    min_j = 0
    max_i = len(matrix) - 1
    max_j = len(matrix[0]) - 1
    
    start_pos_i = i + (i - min_i) if i - num_neighbors < min_i else i - num_neighbors
    start_pos_j = j + (j - min_j) if j - num_neighbors < min_j else j - num_neighbors
    end_pos_i = i + (max_i - i) if i + num_neighbors > max_i else i + num_neighbors
    end_pos_j = j + (max_j - j) if j + num_neighbors > max_j else j + num_neighbors  

    locs = []
    for _i in range(start_pos_i, end_pos_i + 1):
        for _j in range(start_pos_j, end_pos_j + 1): 
            locs.append((_i, _j))
    horizontal = [loc for loc in locs if loc[0]==i]
    vertical = [loc for loc in locs if loc[1]==j]
    diagonal = [loc for loc in locs if loc[0]==loc[1]]
    antidiagonal = [loc for loc in locs if loc[0]+loc[1]==(i+j)]
    return {
        "horizontal": horizontal,
        "vertical": vertical,
        "diagonal": diagonal,
        "antidiagonal": antidiagonal
    }

In [88]:
board_adj = [
    [1, 2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]]

ba = adjacency_check(i=3, j=3, matrix=board_adj)
assert ba["horizontal"] == [(3, 1), (3, 2), (3, 3), (3, 4)]
assert ba["vertical"] == [(1, 3), (2, 3), (3, 3), (4, 3)]
assert ba["diagonal"] == [(1, 1), (2, 2), (3, 3), (4, 4)]
assert ba["antidiagonal"] == [(2, 4), (3, 3), (4, 2)]

In [91]:
from pprint import pprint
pprint(ba)

{'antidiagonal': [(2, 4), (3, 3), (4, 2)],
 'diagonal': [(1, 1), (2, 2), (3, 3), (4, 4)],
 'horizontal': [(3, 1), (3, 2), (3, 3), (3, 4)],
 'vertical': [(1, 3), (2, 3), (3, 3), (4, 3)]}


In [110]:
def horizontal_left_search(i, j, matrix, num_neighbors):
    min_i = 0 
    min_j = 0
    max_i = len(matrix) - 1
    max_j = len(matrix[0]) - 1
    
    xs = []
    for n in range(0, num_neighbors+1):
        _i = i
        _j = j - n
        if _i >= min_i and _j >= min_j and _i <= max_i and _j <= max_j:
            xs.append(matrix[_i][_j])
    return all(item == 1 for item in xs)

board = [
    [ 1,  2,  3,  4,  5],
    [ 6,  7,  8,  9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]]
horizontal_left_search(i=2, j=1, matrix=board, num_neighbors=2)

False

In [112]:
def horizontal_right_search(i, j, matrix, num_neighbors):
    min_i = 0 
    min_j = 0
    max_i = len(matrix) - 1
    max_j = len(matrix[0]) - 1
    
    xs = []
    for n in range(0, num_neighbors+1):
        _i = i
        _j = j + n
        if _i >= min_i and _j >= min_j and _i <= max_i and _j <= max_j:
            xs.append(matrix[_i][_j])
    print(xs)
#     return all(item == 1 for item in xs)

board = [
    [ 1,  2,  3,  4,  5],
    [ 6,  7,  8,  9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]]
horizontal_right_search(i=2, j=1, matrix=board, num_neighbors=2)

[12]
[12, 13]
[12, 13, 14]


In [113]:
def vertical_up_search(i, j, matrix, num_neighbors):
    min_i = 0 
    min_j = 0
    max_i = len(matrix) - 1
    max_j = len(matrix[0]) - 1
    
    xs = []
    for n in range(0, num_neighbors+1):
        _i = i - n
        _j = j
        if _i >= min_i and _j >= min_j and _i <= max_i and _j <= max_j:
            xs.append(matrix[_i][_j])
    print(xs)
#     return all(item == 1 for item in xs)

board = [
    [ 1,  2,  3,  4,  5],
    [ 6,  7,  8,  9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]]
vertical_up_search(i=2, j=1, matrix=board, num_neighbors=2)

[12, 7, 2]


In [114]:
def vertical_down_search(i, j, matrix, num_neighbors):
    min_i = 0 
    min_j = 0
    max_i = len(matrix) - 1
    max_j = len(matrix[0]) - 1
    
    xs = []
    for n in range(0, num_neighbors+1):
        _i = i + n
        _j = j
        if _i >= min_i and _j >= min_j and _i <= max_i and _j <= max_j:
            xs.append(matrix[_i][_j])
    print(xs)
#     return all(item == 1 for item in xs)

board = [
    [ 1,  2,  3,  4,  5],
    [ 6,  7,  8,  9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]]
vertical_down_search(i=2, j=1, matrix=board, num_neighbors=2)

[12, 17, 22]


In [117]:
def diagonal_left_search(i, j, matrix, num_neighbors):
    min_i = 0 
    min_j = 0
    max_i = len(matrix) - 1
    max_j = len(matrix[0]) - 1
    
    xs = []
    for n in range(0, num_neighbors+1):
        _i = i - n
        _j = j - n 
        if _i >= min_i and _j >= min_j and _i <= max_i and _j <= max_j:
            xs.append(matrix[_i][_j])
    print(xs)
#     return all(item == 1 for item in xs)

board = [
    [ 1,  2,  3,  4,  5],
    [ 6,  7,  8,  9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]]
diagonal_left_search(i=2, j=2, matrix=board, num_neighbors=2)

[13, 7, 1]


In [118]:
def diagonal_right_search(i, j, matrix, num_neighbors):
    min_i = 0 
    min_j = 0
    max_i = len(matrix) - 1
    max_j = len(matrix[0]) - 1
    
    xs = []
    for n in range(0, num_neighbors+1):
        _i = i + n
        _j = j + n 
        if _i >= min_i and _j >= min_j and _i <= max_i and _j <= max_j:
            xs.append(matrix[_i][_j])
    print(xs)
#     return all(item == 1 for item in xs)

board = [
    [ 1,  2,  3,  4,  5],
    [ 6,  7,  8,  9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]]
diagonal_right_search(i=2, j=2, matrix=board, num_neighbors=2)

[13, 19, 25]


In [120]:
def antidiagonal_left_search(i, j, matrix, num_neighbors):
    min_i = 0 
    min_j = 0
    max_i = len(matrix) - 1
    max_j = len(matrix[0]) - 1
    
    xs = []
    for n in range(0, num_neighbors+1):
        _i = i - n
        _j = j - n 
        if _i >= min_i and _j >= min_j and _i <= max_i and _j <= max_j:
            xs.append(matrix[_i][_j])
    print(xs)
#     return all(item == 1 for item in xs)

board = [
    [ 1,  2,  3,  4,  5],
    [ 6,  7,  8,  9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]]
antidiagonal_left_search(i=2, j=2, matrix=board, num_neighbors=2)

[13, 7, 1]


In [122]:
def antidiagonal_right_search(i, j, matrix, num_neighbors):
    min_i = 0 
    min_j = 0
    max_i = len(matrix) - 1
    max_j = len(matrix[0]) - 1
    
    xs = []
    for n in range(0, num_neighbors+1):
        _i = i + n
        _j = j + n 
        if _i >= min_i and _j >= min_j and _i <= max_i and _j <= max_j:
            xs.append(matrix[_i][_j])
    print(xs)
#     return all(item == 1 for item in xs)

board = [
    [ 1,  2,  3,  4,  5],
    [ 6,  7,  8,  9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25]]
antidiagonal_right_search(i=2, j=2, matrix=board, num_neighbors=2)

[13, 19, 25]


In [ ]:
def check_middle_for_x_win(i, j, matrix):
        num_neighbors = 1
        
        horizontal = all(horizontal_left_search(i, j, matrix, num_neighbors),
                         horizontal_left_search(i, j, matrix, num_neighbors))
        vertical = all(vertical_up_search(i, j, matrix, num_neighbors),
                        vertical_down_search(i, j, matrix, num_neighbors))
        diagonal = all(diagonal_right_search(i, j, matrix, num_neighbors),
                       diagonal_left_search(i, j, matrix, num_neighbors))
        antidiagonal = all(antidiagonal_right_search(i, j, matrix, num_neighbors),
                        antidiagonal_left_search(i, j, matrix, num_neighbors))
        
        return any(horizontal, vertical, diagonal, antidiagonal)  

In [ ]:
def eval_attacker_move(i, j, matrix):
    return any(
        horizontal_left_search(i, j, matrix, num_neighbors),
        horizontal_right_search(i, j, matrix, num_neighbors),
        vertical_up_search(i, j, matrix, num_neighbors),
        vertical_down_search(i, j, matrix, num_neighbors),
        diagonal_right_search(i, j, matrix, num_neighbors),
        diagonal_left_search(i, j, matrix, num_neighbors),
        antidiagonal_right_search(i, j, matrix, num_neighbors),
        antidiagonal_left_search(i, j, matrix, num_neighbors),   
    )

In [126]:
import itertools
set(itertools.permutations([1, 0, 1]))

{(0, 1, 1), (1, 0, 1), (1, 1, 0)}